# 4.8 Tasks

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\\Users\\Ruf\\Desktop\\Alessia\\Instacart Basket Analysis-Alessia'

In [8]:
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated_v2.pkl'))

## 2. “order_number” column grouped by “department_id” for the entire dataframe.

**check what we have in ords_prods_merge**

In [20]:
ords_prods_merge.head(5)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,department_id,prices,_merge,price_range_loc,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,7,9.0,both,NaN,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,1,1,2398795,1,2,3,7,15.0,196,1,...,7,9.0,both,NaN,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer
2,2,2,473747,1,3,3,12,21.0,196,1,...,7,9.0,both,NaN,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer
3,3,3,2254736,1,4,4,7,29.0,196,1,...,7,9.0,both,NaN,Mid-range product,Least busy,Least busy days,Average orders,10,New customer
4,4,4,431534,1,5,4,15,28.0,196,1,...,7,9.0,both,NaN,Mid-range product,Least busy,Least busy days,Most orders,10,New customer


**find min, max, mean order_number of each department**

In [21]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

## 3. Result differ from the previous excercise. We still have 1 as MIN value, 99 as MAX, but averages are different, which is logic as we are using  a larger dataframe. On top of that, we also have more department_id

## 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

**Split the data into groups based on the “user_id” column; Apply the transform() function on the “order_number” column to generate the maximum orders for each user; Create a new column, “max_order,” into which you’ll place the results of your aggregation.**

In [25]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

**flagging customers depending on max_order**

In [26]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [27]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [28]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

**check the frequency of each loyalty flag**

In [29]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

**check only few columns to see if everything looks fine**

In [31]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(30)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


## 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). 

**using the same loyalty flag, but this time getting price instead of order_number and using .describe function**

In [32]:
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10284093.0,10.386336,328.017787,1.0,4.2,7.4,11.2,99999.0
New customer,6243990.0,13.294670,597.560299,1.0,4.2,7.4,11.3,99999.0
Regular customer,15876776.0,12.495717,539.720919,1.0,4.2,7.4,11.3,99999.0


## Loyal customers seem to be spending less in average vs. New and Regular. But the standard deviation in New and Regular is much higher. This could mean that only few New and Regular made a large purchase, therefore skewing the mean.

# 6. Create a spending flag for each user based on the average price across all their orders

**create a new column with the average prices**

In [34]:
ords_prods_merge['avg_price_order'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

**create the flags and apply them**

In [36]:
ords_prods_merge.loc[ords_prods_merge['avg_price_order'] < 10, 'spending_flag'] = 'Low spender'

In [37]:
ords_prods_merge.loc[ords_prods_merge['avg_price_order'] >= 10, 'spending_flag'] = 'High spender'

**check if it looks fine**

In [41]:
ords_prods_merge.head(10)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,_merge,price_range_loc,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_order,spending_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,NaN,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,1,1,2398795,1,2,3,7,15.0,196,1,...,both,NaN,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender
2,2,2,473747,1,3,3,12,21.0,196,1,...,both,NaN,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender
3,3,3,2254736,1,4,4,7,29.0,196,1,...,both,NaN,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender
4,4,4,431534,1,5,4,15,28.0,196,1,...,both,NaN,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender
5,5,5,3367565,1,6,2,7,19.0,196,1,...,both,NaN,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
6,6,6,550135,1,7,1,9,20.0,196,1,...,both,NaN,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
7,7,7,3108588,1,8,1,14,14.0,196,2,...,both,NaN,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
8,8,8,2295261,1,9,1,16,0.0,196,4,...,both,NaN,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
9,9,9,2550362,1,10,4,8,30.0,196,1,...,both,NaN,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender


**check the frequency of each flagging**

In [42]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

## 7. Determine frequent versus non-frequent customers. 

**create column with the median value of days_since_prior_order**

In [44]:
ords_prods_merge['median_freq'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

**create flags and apply them**

In [45]:
ords_prods_merge.loc[ords_prods_merge['median_freq'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [46]:
ords_prods_merge.loc[(ords_prods_merge['median_freq'] > 10) & (ords_prods_merge['median_freq'] <= 20), 'frequency_flag'] = 'Regular customer'

In [47]:
ords_prods_merge.loc[ords_prods_merge['median_freq'] <= 10, 'frequency_flag'] = 'Frequent customer'

**check the frequency of each flag**

In [48]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [49]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated_v2.pkl'))